# Dataset: *sydneysiege*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(22775, 11)
(1173, 7)


In [4]:
df.isnull().sum()

src_tweet_id           0
src_user_id            0
src_tweet              0
src_date               0
reply_tweet_id         0
reply_user_id          0
reply_tweet            0
reply_date             0
label                  0
cleaned_reply_tw    1418
reply_sentiment        0
dtype: int64

In [7]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)
df.shape

(21357, 11)

In [5]:
src_tw_df.isnull().sum()

src_tweet_id      0
src_user_id       0
src_tweet         0
src_date          0
label             0
cleaned_src_tw    4
src_sentiment     0
dtype: int64

In [14]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

158    RT @martydownunder: If the black flag seems fa...
287    RT RT RT @ZeeGingaNinja: because it seems that...
347    RT @newscomauHQ: Every Australian should be ve...
680    RT @tomsteinfort: Terrifying photo of hostages...
Name: src_tweet, dtype: object

In [32]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw
3966,"If the black flag seems familiar, it's because...",black flag seem familiar also fly gaza protest...
3967,@OrenKessler @FridaGhitis @martydownunder Back...,back flag also use al nusra idf help capture b...
3968,@OrenKessler @martydownunder This flag is popu...,flag popular throughout extreme muslim world s...
3969,@OrenKessler @martydownunder They should have ...,least wash flag
3970,@OrenKessler @paddo_ron @martydownunder So cou...,could anything domestic dispute gaza know
...,...,...
14767,@BlogsofWar @tomsteinfort Hands up Don't shoot?,hand dont shoot
14768,@BlogsofWar @Rekka_K @tomsteinfort correction ...,correction not isis flag still use isis sell n...
14769,@BlogsofWar @tomsteinfort not an ISIS flag. Fo...,not isis flag jhabat al nusra flag sydneycafes...
14770,Neoconservatives right again. The Islamist thr...,neoconservatives right islamist threat real


## Emotion Analysis

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [5]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [6]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [7]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [8]:
df['reply_emo_ro'] = emotion_ro
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df-2.csv',index=False)

In [9]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [10]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [ ]:
print("original reply tw: ", df.reply_tweet[8])
print("cleaned reply tw: ", df.cleaned_reply_tw[8])

print('roBERT based: ',df.reply_emo_ro[8])
print('t5 based: ',df.reply_emo_t5[8])
print('text2emotion: ', df.reply_emo_te[8])